In [ ]:
# Coral Health Classification with YOLOv8 and Optuna (VS Code Version)

# This notebook is adapted for local development in VS Code, not Google Colab.
# It assumes you have the required dataset and dependencies installed locally.

# 1. Install required packages (run in terminal if not already installed)
# !pip install ultralytics optuna

# 2. Import libraries
from ultralytics import YOLO
import optuna
import torch

# Clear GPU memory before starting
torch.cuda.empty_cache()

# 3. Set dataset path
DATASET_PATH = r"C:\Users\modij\Aqua_trace_copy\aqua_trace\NOAA_CORAL_DATASET\NOAA-PIFSC-ESD-CORAL-Bleaching-Dataset"

# 4. Define Optuna objective function
def objective(trial):
    # Clear GPU memory before each trial
    torch.cuda.empty_cache()
    
    # Hyperparameter search space (smaller batch sizes for 8GB GPU)
    batch = trial.suggest_categorical("batch", [8, 16, 32])
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam", "AdamW"])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)

    # Load a pretrained YOLOv8m classification model
    model = YOLO("yolov8m-cls.pt")

    try:
        # Train the model with suggested hyperparameters
        results = model.train(
            data=DATASET_PATH,
            epochs=5,
            imgsz=384,
            batch=batch,
            lr0=lr0,
            optimizer=optimizer,
            weight_decay=weight_decay,
            patience=3,
            erasing=0.1,
            fliplr=0.5,
            hsv_h=0.01,
            hsv_s=0.3,
            hsv_v=0.2,
            project="Optuna_YOLO_CLS",
            name=f"trial_{trial.number}",
            verbose=False,
            workers=4  # Reduce workers to save memory
        )
        # Return Top-1 accuracy for Optuna to maximize
        top1 = getattr(results, 'top1', None)
        return top1 if top1 is not None else 0.0
    except RuntimeError as e:
        if "CUDA" in str(e) or "out of memory" in str(e):
            torch.cuda.empty_cache()
            print(f"CUDA OOM error with batch={batch}, skipping trial")
            return 0.0
        raise e
    finally:
        torch.cuda.empty_cache()

# 5. Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=3)

# 6. Print best trial results
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

c:\Users\modij\anaconda3\envs\torch121\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2026-02-01 11:31:17,402] A new study created in memory with name: no-name-111cd231-9196-41d3-a2d4-b8acddb0189d


Ultralytics 8.4.9  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\modij\Aqua_trace_copy\aqua_trace\NOAA_CORAL_DATASET\NOAA-PIFSC-ESD-CORAL-Bleaching-Dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=5, erasing=0.1, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.3, hsv_v=0.2, imgsz=384, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008114231761244623, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_014

[W 2026-02-01 11:35:06,086] Trial 0 failed with parameters: {'batch': 64, 'lr0': 0.0008114231761244623, 'optimizer': 'AdamW', 'weight_decay': 0.00017225017164532375} because of the following error: RuntimeError('Caught RuntimeError in pin memory thread for device 0.\nOriginal Traceback (most recent call last):\n  File "c:\\Users\\modij\\anaconda3\\envs\\torch121\\lib\\site-packages\\torch\\utils\\data\\_utils\\pin_memory.py", line 41, in do_one_step\n    data = pin_memory(data, device)\n  File "c:\\Users\\modij\\anaconda3\\envs\\torch121\\lib\\site-packages\\torch\\utils\\data\\_utils\\pin_memory.py", line 75, in pin_memory\n    {k: pin_memory(sample, device) for k, sample in data.items()}\n  File "c:\\Users\\modij\\anaconda3\\envs\\torch121\\lib\\site-packages\\torch\\utils\\data\\_utils\\pin_memory.py", line 75, in <dictcomp>\n    {k: pin_memory(sample, device) for k, sample in data.items()}\n  File "c:\\Users\\modij\\anaconda3\\envs\\torch121\\lib\\site-packages\\torch\\utils\\data\

RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "c:\Users\modij\anaconda3\envs\torch121\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 41, in do_one_step
    data = pin_memory(data, device)
  File "c:\Users\modij\anaconda3\envs\torch121\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 75, in pin_memory
    {k: pin_memory(sample, device) for k, sample in data.items()}
  File "c:\Users\modij\anaconda3\envs\torch121\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 75, in <dictcomp>
    {k: pin_memory(sample, device) for k, sample in data.items()}
  File "c:\Users\modij\anaconda3\envs\torch121\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 64, in pin_memory
    return data.pin_memory(device)
RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debuggi